In [1]:
%pip install --upgrade pandas

In [10]:
import pandas as pd

weather = pd.read_csv("all.csv", index_col="name")

In [3]:
weather

,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
name,,,,,,,,,,,,,,,,,,,,,
Bhopal,22-03-2023,31.2,19.0,25.7,29.1,19.0,25.0,7.8,35.1,0.0,...,22.5,10,10,2023-03-22T06:22:59,2023-03-22T18:32:09,0.02,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"VABP,remote"
Bhopal,23-03-2023,32.0,22.0,27.2,29.9,22.0,26.3,7.5,30.0,0.0,...,22.1,9,10,2023-03-23T06:22:01,2023-03-23T18:32:32,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,NaN
Bhopal,24-03-2023,33.9,22.6,28.5,31.8,22.6,27.5,7.7,27.7,0.0,...,24.6,10,10,2023-03-24T06:21:02,2023-03-24T18:32:55,0.08,Clear,Clear conditions throughout the day.,clear-day,NaN
Bhopal,25-03-2023,33.6,21.1,27.8,31.3,21.1,26.8,4.9,23.7,0.0,...,25.7,10,10,2023-03-25T06:20:03,2023-03-25T18:33:17,0.11,Clear,Clear conditions throughout the day.,clear-day,NaN
Bhopal,26-03-2023,34.2,22.0,28.4,31.9,22.0,27.2,4.2,21.8,0.1,...,24.0,10,10,2023-03-26T06:19:04,2023-03-26T18:33:40,0.15,Clear,Clear conditions throughout the day.,clear-day,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pune,01-04-2023,31.1,19.6,25.2,29.6,19.6,24.8,13.9,53.4,0.1,...,19.2,7,10,2023-04-01T06:29:13,2023-04-01T18:48:16,0.34,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,NaN
Pune,02-04-2023,32.9,19.0,25.2,30.8,19.0,24.5,9.7,41.5,0.0,...,24.9,9,10,2023-04-02T06:28:24,2023-04-02T18:48:29,0.37,Clear,Clear conditions throughout the day.,clear-day,NaN
Pune,03-04-2023,33.5,18.1,25.0,31.0,18.1,24.2,6.7,36.5,0.0,...,27.3,10,10,2023-04-03T06:27:35,2023-04-03T18:48:43,0.41,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,NaN


In [4]:
weather.apply(pd.isnull).sum()/weather.shape[0]

datetime            0.000000
tempmax             0.000000
tempmin             0.000000
temp                0.000000
feelslikemax        0.000000
feelslikemin        0.000000
feelslike           0.000000
dew                 0.000000
humidity            0.000000
precip              0.000000
precipprob          0.000000
precipcover         0.000000
preciptype          0.696970
snow                0.000000
snowdepth           0.227273
windgust            0.000000
windspeed           0.000000
winddir             0.000000
sealevelpressure    0.000000
cloudcover          0.000000
visibility          0.000000
solarradiation      0.000000
solarenergy         0.000000
uvindex             0.000000
severerisk          0.000000
sunrise             0.000000
sunset              0.000000
moonphase           0.000000
conditions          0.000000
description         0.000000
icon                0.000000
stations            0.924242
dtype: float64

In [11]:
core_weather = weather[[ "datetime","precip", "snow", "snowdepth", "tempmax", "tempmin"]].copy()
core_weather.columns = ["date","precip", "snow", "snow_depth", "temp_max", "temp_min"]

In [6]:
core_weather.apply(pd.isnull).sum()

precip         0
snow           0
snow_depth    15
temp_max       0
temp_min       0
dtype: int64

In [7]:
core_weather["snow"].value_counts()

0    66
Name: snow, dtype: int64

In [8]:
core_weather["snow_depth"].value_counts()

0.0    51
Name: snow_depth, dtype: int64

In [9]:
del core_weather["snow"]

In [10]:
del core_weather["snow_depth"]

In [11]:
core_weather[pd.isnull(core_weather["precip"])]

,precip,temp_max,temp_min
name,,,


In [12]:
core_weather.loc["22-03-2023":]

,date,precip,snow,snow_depth,temp_max,temp_min
name,,,,,,
Bhopal,22-03-2023,0.0,0,0.0,31.2,19.0
Bhopal,23-03-2023,0.0,0,0.0,32.0,22.0
Bhopal,24-03-2023,0.0,0,0.0,33.9,22.6
Bhopal,25-03-2023,0.0,0,0.0,33.6,21.1
Bhopal,26-03-2023,0.1,0,0.0,34.2,22.0
...,...,...,...,...,...,...
Pune,01-04-2023,0.1,0,0.0,31.1,19.6
Pune,02-04-2023,0.0,0,0.0,32.9,19.0
Pune,03-04-2023,0.0,0,0.0,33.5,18.1
